In [15]:
import pandas as pd
import quandl
import numpy as np
import mplfinance as mpf

apiKey="TmwEsGGvzZL-Y9o2o_KZ"
quandl.ApiConfig.api_key = apiKey
df = quandl.get('CHRIS/CME_ES1', start_date='2010-01-01', end_date='2021-03-12')
df.rename(columns={"Last": "Close","Previous Day Open Interest":"Open Interest"},inplace = True)
df.drop(columns=['Change','Settle'],inplace = True)

In [30]:
def calculate_weekly_zones(df):
	agg_dict = {'Open': 'first',
          'High': 'max',
          'Low': 'min',
          'Close': 'last',
          'Volume': 'sum'}
	df = df.resample('W-Mon',label="left",closed="left").agg(agg_dict)
	df=calculate_zones(df)
	#df=df.resample('1d').pad()
	return df
	
def calculate_zones(df):
	#Setup Zones
	df['C']=(df['Close']+df['Low']+df['High'])/3
	df['range']=df['High']-df['Low']
	df['D']=df['C']+(df['range']/2)
	df['B']=df['C']-(df['range']/2)
	df['E']=df['C']+(df['range'])
	df['A']=df['C']-(df['range'])
	#Shift zones columns
	#a['x2'] = a.x2.shift(1)
	df['C']=df.C.shift(1)
	df['D']=df.D.shift(1)
	df['B']=df.B.shift(1)
	df['E']=df.E.shift(1)
	df['A']=df.A.shift(1)
	df['range']=df.range.shift(1)

	#SET CLOSE AND OPEN ZONES 	
	conditions=[
		(df['Close']<= df['A']),
		(df['Close']<= df['B']) & (df['Close']> df['A']),
		(df['Close']<= df['C']) & (df['Close']> df['B']),
		(df['Close']<= df['D']) & (df['Close']> df['C']),
		(df['Close']<= df['E']) & (df['Close']> df['D']),
		(df['Close']> df['E'])
	]
	choices=['CL1','CL2','CL3','CL4','CL5','CL6']
	df['CLZONE']=np.select(conditions,choices)
	conditions=[
		(df['Open']<= df['A']),
		(df['Open']<= df['B']) & (df['Open']> df['A']),
		(df['Open']<= df['C']) & (df['Open']> df['B']),
		(df['Open']<= df['D']) & (df['Open']> df['C']),
		(df['Open']<= df['E']) & (df['Open']> df['D']),
		(df['Open']> df['E'])
	]
	choices=['OP1','OP2','OP3','OP4','OP5','OP6']
	df['OPZONE']=np.select(conditions,choices)
	df['CLZONE']=df.CLZONE.shift(1)
	df = df.iloc[2:]
	df['CLOP']=df['CLZONE']+df['OPZONE']
	df.drop(columns=['CLZONE','OPZONE'],inplace = True)
	return df

def df_pad_to_daily(df):
	df=df.resample('1d',label="left",closed="left").pad()
	df.rename(columns={"C": "C_W","A": "A_W","B": "B_W","D": "D_W","E": "E_W",},inplace = True)
	df.rename(columns={'CLOP':'CLOP_W'},inplace=True)
	return df

def correct_prices_in_weekly(df_fr_W,df):
	df_fr_W["Open"]=df["Open"]
	df_fr_W["High"]=df["High"]
	df_fr_W["Low"]=df["Low"]
	df_fr_W["Close"]=df["Close"]
	df_fr_W["Volume"]=df["Volume"]
	df_fr_W = df_fr_W.dropna()
	return df_fr_W


In [72]:
df_weekly=calculate_weekly_zones(df)
df_fr_W=df_pad_to_daily(df_weekly)
df_fr_W=correct_prices_in_weekly(df_fr_W,df)
df_fr_W=calculate_zones(df_fr_W)
df=calculate_zones(df)

<ipython-input-30-03d961293c5f>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CLOP']=df['CLZONE']+df['OPZONE']
K:\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-30-03d961293c5f>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [74]:
df.head()

,Open,High,Low,Close,Volume,Open Interest,C,range,D,B,E,A,CLOP
Date,,,,,,,,,,,,,
2010-01-06,1132.00,1135.50,1127.25,1133.0,1259921.0,2396493.0,1130.083333,8.00,1134.083333,1126.083333,1138.083333,1122.083333,CL5OP4
2010-01-07,1133.00,1138.75,1127.00,1137.5,1567025.0,2406352.0,1131.916667,8.25,1136.041667,1127.791667,1140.166667,1123.666667,CL4OP4
2010-01-08,1137.25,1141.75,1131.00,1141.5,1527666.0,2420194.0,1134.416667,11.75,1140.291667,1128.541667,1146.166667,1122.666667,CL5OP4
2010-01-11,1141.75,1148.00,1137.75,1142.5,1456443.0,2427453.0,1138.083333,10.75,1143.458333,1132.708333,1148.833333,1127.333333,CL5OP4
2010-01-12,1141.25,1143.00,1127.75,1134.0,2117166.0,2439036.0,1142.750000,10.25,1147.875000,1137.625000,1153.000000,1132.500000,CL4OP3


In [75]:
df_weekly.head()

,Open,High,Low,Close,Volume,C,range,D,B,E,A,CLOP
Date,,,,,,,,,,,,
2010-01-18,1131.75,1147.00,1086.25,1091.00,13362600.0,1135.916667,20.50,1146.166667,1125.666667,1156.416667,1115.416667,CL4OP3
2010-01-25,1094.00,1103.50,1066.50,1070.50,13284325.0,1108.083333,60.75,1138.458333,1077.708333,1168.833333,1047.333333,CL1OP3
2010-02-01,1070.00,1101.50,1040.75,1059.75,13170284.0,1080.166667,37.00,1098.666667,1061.666667,1117.166667,1043.166667,CL2OP3
2010-02-08,1062.00,1079.75,1053.00,1079.00,12289657.0,1067.333333,60.75,1097.708333,1036.958333,1128.083333,1006.583333,CL2OP3
2010-02-15,1077.50,1111.00,1073.50,1106.25,10108872.0,1070.583333,26.75,1083.958333,1057.208333,1097.333333,1043.833333,CL4OP4


In [76]:
df_fr_W.head()

,Open,High,Low,Close,Volume,C_W,range,D_W,B_W,E_W,A_W,CLOP_W,C,D,B,E,A,CLOP
Date,,,,,,,,,,,,,,,,,,
2010-01-20,1145.00,1146.50,1125.25,1134.25,2391710.0,1135.916667,20.75,1146.166667,1125.666667,1156.416667,1115.416667,CL4OP3,1139.666667,1150.041667,1129.291667,1160.416667,1118.916667,CL6OP4
2010-01-21,1134.50,1139.25,1110.25,1110.50,3438955.0,1135.916667,21.25,1146.166667,1125.666667,1156.416667,1115.416667,CL4OP3,1135.333333,1145.958333,1124.708333,1156.583333,1114.083333,CL3OP3
2010-01-22,1109.75,1115.75,1086.25,1091.00,3544446.0,1135.916667,29.00,1146.166667,1125.666667,1156.416667,1115.416667,CL4OP3,1120.000000,1134.500000,1105.500000,1149.000000,1091.000000,CL1OP3
2010-01-25,1094.00,1103.00,1089.75,1092.50,2080292.0,1108.083333,29.50,1138.458333,1077.708333,1168.833333,1047.333333,CL1OP3,1097.666667,1112.416667,1082.916667,1127.166667,1068.166667,CL1OP3
2010-01-26,1094.00,1100.00,1081.00,1087.00,2449263.0,1108.083333,13.25,1138.458333,1077.708333,1168.833333,1047.333333,CL1OP3,1095.083333,1101.708333,1088.458333,1108.333333,1081.833333,CL3OP3


In [92]:
#Given a time series with zones returns 2 probability matrixes one for zones and one for bands
def calculate_probs(df):
	# Zones Reached
	conditions=[
		(df['Low']<df['A'])
	]
	choices=[1]
	df['Reach1']=np.select(conditions,choices)
	conditions=[
		(df['High']>=df['B']) & (df['Low']<=df['A']),
		(df['Low']<df['B']) & (df['Low']>=df['A']),
		(df['High']<df['B']) & (df['High']>=df['A'])
	]
	choices=[1,1,1]
	df['Reach2']=np.select(conditions,choices)
	conditions=[
		(df['High']>=df['C']) & (df['Low']<=df['B']),
		(df['Low']<df['C']) & (df['Low']>=df['B']),
		(df['High']<df['C']) & (df['High']>=df['B'])
	]
	choices=[1,1,1]
	df['Reach3']=np.select(conditions,choices)
	conditions=[
		(df['High']>=df['D']) & (df['Low']<=df['C']),
		(df['Low']<df['D']) & (df['Low']>=df['C']),
		(df['High']<df['D']) & (df['High']>=df['C'])
	]
	choices=[1,1,1]
	df['Reach4']=np.select(conditions,choices)
	conditions=[
		(df['High']>=df['E']) & (df['Low']<=df['D']),
		(df['Low']<df['E']) & (df['Low']>=df['D']),
		(df['High']<df['E']) & (df['High']>=df['D'])
	]
	choices=[1,1,1]
	df['Reach5']=np.select(conditions,choices)
	conditions=[
		(df['High']>df['E'])
	]
	choices=[1]
	df['Reach6']=np.select(conditions,choices)
	# Zonas como Resistencia:
	#
	conditions=[
		(df['High']>df['A']) & (df['High']<(df['B']))
	]
	choices=[1]
	df['RES2']=np.select(conditions,choices)
	conditions=[
		(df['High']>df['B']) & (df['High']<(df['C']))
	]
	choices=[1]
	df['RES3']=np.select(conditions,choices)
	conditions=[
		(df['High']>df['C']) & (df['High']<(df['D']))
	]
	choices=[1]
	df['RES4']=np.select(conditions,choices)
	conditions=[
		(df['High']>df['D']) & (df['High']<(df['E']))
	]
	choices=[1]
	df['RES5']=np.select(conditions,choices)
	conditions=[
		(df['High']>df['E']) & (df['Close']<(df['E']))
	]
	choices=[1]
	df['RES6']=np.select(conditions,choices)
	# Zonas como Soporte:
	#
	conditions=[
		(df['Low']<(df['A'])) & (df['Close']>df['A'])
	]
	choices=[1]
	df['SUP1']=np.select(conditions,choices)
	conditions=[
		(df['Low']<(df['B'])) & (df['Low']>df['A'])
	]
	choices=[1]
	df['SUP2']=np.select(conditions,choices)
	conditions=[
		(df['Low']<(df['C'])) & (df['Low']>df['B'])
	]
	choices=[1]
	df['SUP3']=np.select(conditions,choices)
	conditions=[
		(df['Low']<(df['D'])) & (df['Low']>df['C'])
	]
	choices=[1]
	df['SUP4']=np.select(conditions,choices)
	conditions=[
		(df['Low']<(df['E'])) & (df['Low']>df['D'])
	]
	choices=[1]
	df['SUP5']=np.select(conditions,choices)
	# Bandas como zonas de soporte y resistencia
	#
	conditions=[
		(df['Low'] >= (df['C']-(df['range']*0.618))) & (df['Low']<= df['B'])
	]
	choices=[1]
	df['SB1']=np.select(conditions,choices)
	conditions=[
		(df['Low']>=(df['C']-(df['range']*1.382))) & (df['Low']<=df['A']) & (df['Close']>=df['A'])
	]
	choices=[1]
	df['SB2']=np.select(conditions,choices)
	conditions=[
		(df['High']<=(df['C']+(df['range']*0.618))) & (df['High']>=df['D']) 
	]
	choices=[1]
	df['RB1']=np.select(conditions,choices)
	conditions=[
		(df['High']<=(df['C']+(df['range']*1.382))) & (df['High']>=df['E']) & (df['Close']<=df['E']) 
	]
	choices=[1]
	df['RB2']=np.select(conditions,choices)
	#############
	#	Creacion de matriz de probabilidades
	###########
	CLOP_combo=df['CLOP'].sort_values().unique()
	prob=pd.DataFrame(index=CLOP_combo,columns=['count','Reach1','Reach2','Reach3','Reach4','Reach5','Reach6','RES2','RES3','RES4','RES5','RES6','SUP1','SUP2','SUP3','SUP4','SUP5', 'SB1', 'SB2', 'RB1', 'RB2'])
	for i in CLOP_combo:
		data=df[df['CLOP']==i]
		probs_raw=data[['Reach1','Reach2','Reach3','Reach4','Reach5','Reach6','RES2','RES3','RES4','RES5','RES6','SUP1','SUP2','SUP3','SUP4','SUP5','SB1', 'SB2', 'RB1', 'RB2']]
		prob.loc[i,"count"]=data.shape[0]
		for c in probs_raw.columns:
			aux=probs_raw[c].value_counts(normalize=True)     
			if(1 in aux.index):
				prob.loc[i,c]=aux[1]*100.0
			else:
				prob.loc[i,c]=0.0
			if((c in ['RES1','SUP1','SB2']) & (prob.loc[i,'Reach1']!= 0)):
				prob.loc[i,c]=prob.loc[i,c]/prob.loc[i,'Reach1']*100.0
			if((c in ['RES2','SUP2','SB1']) & (prob.loc[i,'Reach2']!= 0)):
				prob.loc[i,c]=prob.loc[i,c]/prob.loc[i,'Reach2']*100.0
			if((c in ['RES3','SUP3']) & (prob.loc[i,'Reach3']!= 0)):
				prob.loc[i,c]=prob.loc[i,c]/prob.loc[i,'Reach3']*100.0
			if((c in ['RES4','SUP4']) & (prob.loc[i,'Reach4']!= 0)):
				prob.loc[i,c]=prob.loc[i,c]/prob.loc[i,'Reach4']*100.0
			if((c in ['RES5','SUP5','RB1']) & (prob.loc[i,'Reach5']!= 0)):
				prob.loc[i,c]=prob.loc[i,c]/prob.loc[i,'Reach5']*100.0
			if((c in ['RES6','SUP6','RB2']) & (prob.loc[i,'Reach6']!= 0)):
				prob.loc[i,c]=prob.loc[i,c]/prob.loc[i,'Reach6']*100.0
	prob.round(2)
	#############
	#	Creacion de matriz de probabilidades Estrictas para bandas
	###########
	prob_strict=pd.DataFrame(index=CLOP_combo,columns=['RB1_s','RB2_s','SB1_s','SB2_s'])
	prob_strict['RB1_s'] = np.divide(prob['RB1']*100, prob['RES5'], out=np.zeros_like(prob['RB1']), where=prob['RES5']!=0)
	prob_strict['RB2_s'] = np.divide(prob['RB2']*100, prob['RES6'], out=np.zeros_like(prob['RB2']), where=prob['RES6']!=0)
	prob_strict['SB1_s'] = np.divide(prob['SB1']*100, prob['SUP2'], out=np.zeros_like(prob['SB1']), where=prob['SUP2']!=0)
	prob_strict['SB2_s'] = np.divide(prob['SB2']*100, prob['SUP1'], out=np.zeros_like(prob['SB2']), where=prob['SUP1']!=0)
	return prob, prob_strict

In [79]:
prob,prob_s=calculate_probs(df)

<ipython-input-78-7734e64575ff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Reach1']=np.select(conditions,choices)
<ipython-input-78-7734e64575ff>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Reach2']=np.select(conditions,choices)
<ipython-input-78-7734e64575ff>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [82]:
prob_w,prob_w_s=calculate_probs(df_weekly)

In [90]:
df[-1:]

,Open,High,Low,Close,Volume,Open Interest,C,range,D,B,...,RES6,SUP1,SUP2,SUP3,SUP4,SUP5,SB1,SB2,RB1,RB2
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-12,3937.5,3947.5,3910.25,3942.0,1416883.0,2264419.0,3928.833333,65.5,3961.583333,3896.083333,...,0,0,0,1,0,0,0,0,0,0


In [94]:
df_fr_W
df_fr_W.rename(columns={"C": "C_D","A": "A_D","B": "B_D","D": "D_D","E": "E_D",},inplace = True)
df_fr_W.rename(columns={"C_W": "C","A_W": "A","B_W": "B","D_W": "D","E_W": "E",},inplace = True)
df_fr_W.rename(columns={'CLOP_W':'CLOP'},inplace=True)

In [96]:
prob_d_w,prob_d_w_s=calculate_probs(df_fr_W)

In [99]:
df_fr_W.rename(columns={'CLOP':'CLOP_W'},inplace=True)
df_fr_W.rename(columns={'CLOP_D':'CLOP'},inplace=True)

In [104]:
prob_d_w,prob_d_w_s=calculate_probs(df_fr_W)

In [117]:
prob_d_w.loc["CL4OP3"],prob.loc["CL4OP3"],prob_w.loc["CL3OP4"]

(count         259
 Reach1     3.4749
 Reach2    7.33591
 Reach3    44.0154
 Reach4    86.4865
 Reach5      38.61
 Reach6    7.72201
 RES2      21.0526
 RES3      7.01754
 RES4      63.8393
 RES5           84
 RES6           35
 SUP1      33.3333
 SUP2      52.6316
 SUP3      86.8421
 SUP4      52.2321
 SUP5           19
 SB1             0
 SB2             0
 RB1             0
 RB2             0
 Name: CL4OP3, dtype: object,
 count         261
 Reach1    27.2031
 Reach2     62.069
 Reach3        100
 Reach4    93.1034
 Reach5    48.2759
 Reach6    19.5402
 RES2            0
 RES3      6.89655
 RES4      47.7366
 RES5      57.9365
 RES6      52.9412
 SUP1      53.5211
 SUP2      55.5556
 SUP3       37.931
 SUP4            0
 SUP5            0
 SB1       16.6667
 SB2       35.2113
 RB1       20.6349
 RB2       37.2549
 Name: CL4OP3, dtype: object,
 count          39
 Reach1    5.12821
 Reach2    38.4615
 Reach3    84.6154
 Reach4        100
 Reach5    64.1026
 Reach6    17.9487
 RES2    

In [116]:
prob_w

,count,Reach1,Reach2,Reach3,Reach4,Reach5,Reach6,RES2,RES3,RES4,...,RES6,SUP1,SUP2,SUP3,SUP4,SUP5,SB1,SB2,RB1,RB2
CL1OP2,1,100,100,100,100,0,0,0,0,100,...,0,100,0,0,0,0,0,100,0,0
CL1OP3,37,16.2162,59.4595,100,72.973,32.4324,8.10811,0,27.027,55.5556,...,33.3333,66.6667,72.7273,40.5405,0,0,31.8182,33.3333,33.3333,33.3333
CL1OP4,4,25,50,100,100,50,50,0,0,50,...,0,100,50,50,0,0,0,100,0,0
CL2OP2,2,50,100,100,100,0,0,0,0,100,...,0,100,50,0,0,0,0,100,0,0
CL2OP3,46,19.5652,58.6957,100,80.4348,50,4.34783,0,19.5652,37.8378,...,50,44.4444,66.6667,41.3043,0,0,22.2222,44.4444,39.1304,50
CL2OP4,5,40,40,60,100,40,20,0,0,60,...,0,50,0,33.3333,40,0,0,0,0,0
CL3OP1,1,100,100,0,0,0,0,100,0,0,...,0,100,0,0,0,0,0,0,0,0
CL3OP2,3,66.6667,100,66.6667,33.3333,0,0,33.3333,50,100,...,0,0,33.3333,0,0,0,0,0,0,0
CL3OP3,64,28.125,65.625,100,87.5,40.625,6.25,0,12.5,53.5714,...,50,50,57.1429,34.375,0,0,14.2857,33.3333,26.9231,50
CL3OP4,39,5.12821,38.4615,84.6154,100,64.1026,17.9487,0,0,35.8974,...,28.5714,100,86.6667,54.5455,15.3846,0,33.3333,0,12,28.5714
